<a href="https://colab.research.google.com/github/tmckim/materials-fa23-colab/blob/main/lectures/lec10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Before you start - Save this notebook!

When you open a new Colab notebook from the WebCampus (like you hopefully did for this one), you cannot save changes. So it's  best to store the Colab notebook in your personal drive `"File > Save a copy in drive..."` **before** you do anything else.

The file will open in a new tab in your web browser, and it is automatically named something like: "**Copy of lec10.ipynb**". You can rename this to just the title of the assignment "**lec10.ipynb**". Make sure you do keep an informative name (like the name of the assignment) so that you know which files to submit back to WebCampus for grading! More instructions on this are at the end of the notebook.


**Where does the notebook get saved in Google Drive?**

By default, the notebook will be copied to a folder called “Colab Notebooks” at the root (home directory) of your Google Drive. If you use this for other courses or personal code notebooks, I recommend creating a folder for this course and then moving the assignments AFTER you have completed them.

In [ ]:
# Setup and add files needed to gdrive
# If you restart colab, start by rerunning this cell first!
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

#!mkdir -p '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/'
%cd /content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/
!rm -r materials-fa23-colab

!git clone https://github.com/tmckim/materials-fa23-colab '/content/gdrive/My Drive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/'

%cd /content/gdrive/MyDrive/colab-materials-NS499DataSci-notebooks/materials-fa23-colab/lectures/

In [ ]:
# Import packages and other things needed
# Don't change this cell; Just run this cell
# If you restart colab, make sure to run this cell again after the first one above^

from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Comparing Two Samples

In [ ]:
# Read in table of data
births = Table.read_table('baby.csv')

In [ ]:
# birth weight in ounces
# days in womb
# age of mother
# height of mother
# weight of mother- pounds
# mother smoking status

In [ ]:
births

In [ ]:
# Select subset of data we will work with
smoking_and_birthweight = births.select('Maternal Smoker', 'Birth Weight')

In [ ]:
# Find how many individuals in each category
...

In [ ]:
# Plot the distributions of the data
smoking_and_birthweight.hist('Birth Weight')

# Test Statistic

Question: What values of our statistic are in favor of the alternative: positive or negative?

In [ ]:
# Return averages
means_table = smoking_and_birthweight.group('Maternal Smoker', np.average)
means_table

In [ ]:
# Use the column we want and calculate the difference
means = means_table.column(1)
observed_difference = means.item(1) - means.item(0)
observed_difference

In [ ]:
# Define a function using what we already did above so we can easily repeat this process many times
def difference_of_means(table, numeric_label, category_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups"""
    
    #table with the two relevant columns
    reduced = ...
    
    # table containing group means
    means_table = ...

    # array of group means
    means = ...
    
    return means.item(1) - means.item(0)

In [ ]:
# Run the function
# Function inputs:
    # table = births
    # numeric_label = Birth Weight column
    # category_label = Maternal Smoker column
difference_of_means(births, 'Birth Weight', 'Maternal Smoker')

Now we need to simulate. But how do we do this? <br>
Simulate another set of people like this, but assume the distributions are the same between the weight differences of the groups. <br>
Seems complicated, how do we do this? <br>
We don't know the sample proportions like we did in the past- we had the population data in the previous examples. We don't have that info here.

# Random Permutation (Shuffling)

In [ ]:
# An example table
letters = Table().with_column('Letter', make_array('a', 'b', 'c', 'd', 'e'))

In [ ]:
# Shuffles the entire table, but you can get repeats
letters.sample()

In [ ]:
# Ensure we get unique permutation
letters.sample(with_replacement = False)

In [ ]:
# Show them together
letters.with_column('Shuffled', letters.sample(with_replacement = False).column(0))

# Simulation Under Null Hypothesis

In [ ]:
# Here is our table again
smoking_and_birthweight

In [ ]:
# Shuffle Maternal Smoker labels, but keep the Birth Weights the same
shuffled_labels = smoking_and_birthweight.sample(with_replacement=False).column('Maternal Smoker')

In [ ]:
# Create new table that contains original and new shuffled categories
original_and_shuffled = smoking_and_birthweight.with_column(
    'Shuffled Label', shuffled_labels
)

In [ ]:
# Check it out
original_and_shuffled

In [ ]:
# Now we can call the function from above, but use the shuffled label instead of original
difference_of_means(original_and_shuffled, 'Birth Weight', 'Shuffled Label')

In [ ]:
# Compare to running it with our original table values
difference_of_means(original_and_shuffled, 'Birth Weight', 'Maternal Smoker')

# Permutation Test

In [ ]:
# Function for our simulation and calculate the test statistic
def one_simulated_difference(table, numeric_label, category_label):
    """Takes: name of table, column label of numerical variable,
    column label of group-label variable
    Returns: Difference of means of the two groups after shuffling labels"""
    
    # array of shuffled labels
    shuffled_labels = table.sample(with_replacement = False).column(category_label)
    
    # table of numerical variable and shuffled labels
    shuffled_table = table.select(numeric_label).with_column('Shuffled Label', shuffled_labels)  # this is our simulated data
    
    return difference_of_means(shuffled_table, numeric_label, 'Shuffled Label')   # computes and returns the test statistic

In [ ]:
# Test the function
one_simulated_difference(births, 'Birth Weight', 'Maternal Smoker')

In [ ]:
# Final step: repeat this many times using a for loop
differences = make_array()

for i in np.arange(2500):
    new_difference = one_simulated_difference(births, 'Birth Weight', 'Maternal Smoker')
    differences = np.append(differences, new_difference)

In [ ]:
# Create a table from the data array and plot the distribution
diff_tbl = Table().with_column('Simulated Differences', differences)
diff_tbl.hist()

print('Observed Difference:', observed_difference)
plots.title('Prediction Under the Null Hypothesis');

In [ ]:
# Calculating the p-value

diff_tbl.where('Simulated Differences', are.below(observed_difference)).num_rows / 2500

Does this imply causation?

In [ ]:
# Review other values in the table
births.group('Maternal Smoker', np.average)

# Randomized Control Experiment

In [ ]:
# Botox treatment for chronic pain
# 1 - improvement in outcome (pain)
# 0 - no change in outcome (pain)
botox = Table.read_table('bta.csv')
botox.show()

In [ ]:
# What are the unique combinations of all of these values
...

In [ ]:
# Average value of improvement for the outcome variable
botox.group('Group', np.average)

# Testing the Hypothesis

In [ ]:
# Use the function we defined above to calculate the difference
# We have a different table, and input columns
observed_diff = difference_of_means(botox, 'Result', 'Group')
observed_diff

In [ ]:
# Run our permutation function
one_simulated_difference(botox, 'Result', 'Group')

In [ ]:
# Simulate many times
simulated_diffs = make_array()

for i in np.arange(10000):
    sim_diff = one_simulated_difference(botox, 'Result', 'Group')
    simulated_diffs = np.append(simulated_diffs, sim_diff)

In [ ]:
# Create a table and plot the distribution
col_name = 'Distance between groups'
Table().with_column(col_name, simulated_diffs).hist(col_name)

plots.scatter(observed_diff, -0.03,c='red')

In [ ]:
# Calculate p-value- threshold is 1% (0.01)
sum(simulated_diffs >= observed_diff)/len(simulated_diffs)